In [ ]:
import fastai
from fastai.vision.all import *
from pathlib import Path
from glob import glob
import cv2
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches

In [ ]:
df = pd.read_csv("../input/uw-madison-gi-tract-image-segmentation/train.csv", low_memory = False)
display(df.head())

In [ ]:
df["case_no"] = df["id"].apply(lambda x: x.split("_")[0]) 

In [ ]:
df["case_no_int"] = df["case_no"].apply(lambda x: x.replace("case", ""))
df["day_no"] = df["id"].apply(lambda x: x.split("_")[1])
display(df.head())

In [ ]:
ss_df = pd.read_csv("../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv")

In [ ]:
DEBUG=len(ss_df)==0
TRAIN_DIR = "../input/uw-madison-gi-tract-image-segmentation/train"
TEST_DIR = "../input/uw-madison-gi-tract-image-segmentation/test"
all_train_images = glob(os.path.join(TRAIN_DIR, "**", "*.png"), recursive = True)
all_test_images = glob(os.path.join(TEST_DIR, "**", "*.png"), recursive = True)


if DEBUG:
    TEST_DIR = TRAIN_DIR
    all_test_images = all_train_images
    ss_df = df.iloc[:10]
    ss_df = ss_df[["id", "class"]]
    ss_df["predicted"] = ""

In [ ]:
df["day_no_int"] = df["day_no"].apply(lambda x: x.replace("day", ""))
df["slice_id"] = df["id"].apply(lambda x: x.split("_")[2:])
df["slice_id"] = df["slice_id"].apply(lambda x: x[0] + "_" + x[1])
display(df.head())

In [ ]:
all_train_images = glob(os.path.join(TRAIN_DIR, "**", "*.png"), recursive = True)

In [ ]:
all_train_images[:5]

In [ ]:
all_train_images[0].rsplit("/", 4)[0]

In [ ]:
df["_partial_ident"] = (all_train_images[0].rsplit("/", 4)[0] + "/" + df["case_no"] + "/" + df["case_no"] + "_" + df["day_no"] + "/scans/" + df["slice_id"])

In [ ]:
_tmp_merge_df = pd.DataFrame({"_partial_ident" : [x.rsplit("_", 4)[0] for x in all_train_images],
                              "f_path" : all_train_images})

In [ ]:
df = df.merge(_tmp_merge_df, on = "_partial_ident").drop(columns = ["_partial_ident"])
display(df.head())

In [ ]:
df["slice_h"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[1]))
df["slice_w"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[2]))

df["px_spacing_h"] = df["f_path"].apply(lambda x: float(x[:-4].rsplit("_", 4)[3]))
df["px_spacing_w"] = df["f_path"].apply(lambda x: float(x[:-4].rsplit("_", 4)[4]))

In [ ]:
l_bowel_df = df[df["class"] == "large_bowel"][["id", "segmentation"]].rename(columns = {"segmentation" : "large_bowel_seg"})
s_bowel_df = df[df["class"] == "small_bowel"][["id", "segmentation"]].rename(columns = {"segmentation" : "small_bowel_seg"})
stomach_df = df[df["class"] == "stomach"][["id", "segmentation"]].rename(columns = {"segmentation" : "stomach_seg"})

df = df.merge(l_bowel_df, on = "id", how = "left")
df = df.merge(s_bowel_df, on = "id", how = "left")
df = df.merge(stomach_df, on = "id", how = "left")

df = df.drop_duplicates(subset = "id").reset_index(drop = True)
df["large_bowel_seg_flag"] = df["large_bowel_seg"].apply(lambda x: not pd.isna(x))
df["small_bowel_seg_flag"] = df["small_bowel_seg"].apply(lambda x: not pd.isna(x))
df["stomach_seg_flag"] = df["stomach_seg"].apply(lambda x: not pd.isna(x))

df["n_segs"] = df["large_bowel_seg_flag"].astype(int) + df["small_bowel_seg_flag"].astype(int) + df["stomach_seg_flag"].astype(int)

In [ ]:
len(df)

In [ ]:
display(ss_df.head())

In [ ]:
slice_px_map = {}
for _, row in df.groupby(['slice_h', 'slice_w', 'px_spacing_h', 'px_spacing_w', 'slice_id'])[["large_bowel_seg", "small_bowel_seg", "stomach_seg"]].first().reset_index().iterrows():
    slice_px_map[f"{row['slice_h']}-{row['slice_w']}-{row['px_spacing_h']}-{row['px_spacing_w']}-{row['slice_id']}-large_bowel"] = row["large_bowel_seg"]
    slice_px_map[f"{row['slice_h']}-{row['slice_w']}-{row['px_spacing_h']}-{row['px_spacing_w']}-{row['slice_id']}-small_bowel"] = row["small_bowel_seg"]
    slice_px_map[f"{row['slice_h']}-{row['slice_w']}-{row['px_spacing_h']}-{row['px_spacing_w']}-{row['slice_id']}-stomach"] = row["stomach_seg"]

In [ ]:
ss_df["case_no"] = ss_df["id"].apply(lambda x: x.split("_")[0]) 
ss_df["case_no_int"] = ss_df["case_no"].apply(lambda x: x.replace("case", ""))
ss_df["day_no"] = ss_df["id"].apply(lambda x: x.split("_")[1])
ss_df["day_no_int"] = ss_df["day_no"].apply(lambda x: x.replace("day", ""))
ss_df["slice_id"] = ss_df["id"].apply(lambda x: x.split("_")[2:])
ss_df["slice_id"] = ss_df["slice_id"].apply(lambda x: x[0] + "_" + x[1])
ss_df["_partial_ident"] = (all_test_images[0].rsplit("/", 4)[0] + "/" + ss_df["case_no"] + "/" + ss_df["case_no"] + "_" + ss_df["day_no"] + "/scans/" + ss_df["slice_id"])
_tmp_merge_df_ss = pd.DataFrame({"_partial_ident" : [x.rsplit("_", 4)[0] for x in all_test_images],
                              "f_path" : all_test_images})

ss_df = ss_df.merge(_tmp_merge_df_ss, on = "_partial_ident").drop(columns = ["_partial_ident"])
display(ss_df.head())

In [ ]:
ss_df["slice_h"] = ss_df["f_path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[1]))
ss_df["slice_w"] = ss_df["f_path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[2]))

ss_df["px_spacing_h"] = ss_df["f_path"].apply(lambda x: float(x[:-4].rsplit("_", 4)[3]))
ss_df["px_spacing_w"] = ss_df["f_path"].apply(lambda x: float(x[:-4].rsplit("_", 4)[4]))

In [ ]:
ss_df["ident"] = ss_df["slice_h"].astype(str)+"-"+ss_df["slice_w"].astype(str)+"-"+ss_df["px_spacing_h"].astype(str)+"-"+ss_df["px_spacing_w"].astype(str)+"-"+ss_df["slice_id"].astype(str)+"-"+ss_df["class"].astype(str)

In [ ]:
ss_df.head()

In [ ]:
ss_df["predicted"] = ss_df["ident"].map(slice_px_map)

In [ ]:
ss_df = ss_df[["id", "class", "predicted"]]
ss_df.to_csv("submission.csv", index = False)
display(ss_df.head())